In [ ]:
from scipy.signal import savgol_filter
data_p = pd.read_csv("4hour.csv")


def denoise(df):
    closing_prices = df['close'].values
    denoised_prices = savgol_filter(closing_prices, window_length=100, polyorder=5)
    df['savgol'] = denoised_prices
    
    return df

data_p = denoise(data_p)
############################
def calculate_moving_averages(df, column='close', short_window=10, long_window=2400):
    
    df['SMA_Short'] = df['savgol'].rolling(window=short_window).mean()
    df['SMA_Long'] = df['savgol'].rolling(window=long_window).mean()
    return df
short_window=20
long_window=50
calculate_moving_averages(data_p)
############################

def generate_signals(df):
    df['Signal'] = 0
    df['Signal'][short_window:] = np.where(df['SMA_Short'][short_window:] > df['SMA_Long'][short_window:], 1, -1)
    
    stoploss = 0.135
    takeprofit = 0
    
    signalsarray = df.Signal.to_numpy()
    close = df.close.to_numpy()
    finalsignals = []
    
    capital = 100000

    initialcap = capital
    
    current = 0
    portfolio = []
    stock = 0
    tradetype = []
    entryidx = []
    exitidx = []
############################


    for i in range(len(df)):
        signal = signalsarray[i]
        
        if current == 0:
            
            portfolio.append(capital)
            
            if signal == 0:
                finalsignals.append(0) ##
                pass
                
            if signal == 1:
                entryidx.append(i)
                # tradetype.append("Long")
                stock = capital/close[i]
                capital=0
                current = 1
                finalsignals.append(p_buy) ##
                
                
            if signal == -1:
                entryidx.append(i)
                # tradetype.append("Short")
                stock = capital/close[i]
                # capital = 2*capital
                current = -1
                finalsignals.append(p_sell) ##
            
        
        
        elif current == 1:
            
            if signal == 1 or signal == 0:
                portfolio.append(stock * close[i])
                finalsignals.append(0) ##
                if len(portfolio[entryidx[-1]:-1]) >= 2:
                    if max(portfolio[entryidx[-1]:-1])*(1-stoploss)>=portfolio[-1]:                  # trailing condition
                        finalsignals[-1] = p_sell ##
                        # print("stoploss, going 1->0 entryidx", entryidx[-1], "trade no. ", len(entryidx))
                        exitidx.append(i)
                        capital = stock*close[i] 
                
                        # brokerage = broker_charge*capital
                        # capital -= brokerage
                        # total_brokerage+=brokerage
                        
                        portfolio.append(capital)
                        # pl = stock * (close[i] - close[entryidx[-1]]) 
                        # gprofitloss.append(pl)
                        # nprofitloss.append(pl-brokerage)
                        current = 0
                        stock=0
                    
            if signal == -1:
                finalsignals.append(p_sell) ##
                
                exitidx.append(i)
                capital = stock*close[i]
                
                # brokerage = broker_charge*capital
                # capital -= brokerage
                # total_brokerage+=brokerage
                
                portfolio.append(capital)
                # pl = stock * (close[i] - close[entryidx[-1]]) 
                # gprofitloss.append(pl)
                # nprofitloss.append(pl-brokerage)
                current = 0
                stock=0
        
        
        
        elif current == -1:

            if signal == -1 or signal == 0:
                
                portfolio.append(capital + stock * (-close[i] + close[entryidx[-1]]))
                finalsignals.append(0) ##
                if len(portfolio[entryidx[-1]:-1]) >= 2:
                    if np.max(portfolio[entryidx[-1]:-1])*(1-stoploss)>=portfolio[-1]:     # trailing condition
                        finalsignals[-1] = p_buy ##
                    
                        print("stoploss, going -1->0 entryidx", entryidx[-1], "trade no. ", len(entryidx))
                        exitidx.append(i)
                        # capital = stock*close[i]
                        
                        # brokerage = broker_charge*capital
                        # capital -= brokerage
                        # total_brokerage+=brokerage
                        
                        capital += stock*close[entryidx[-1]] - stock*close[i]
            
                        # pl = stock * (close[entryidx[-1]] - close[i])
                        # gprofitloss.append(pl)
                        # nprofitloss.append(pl-brokerage)
                        current = 0
                        stock=0
                    
            
            if signal == +1:
                finalsignals.append(p_buy)
                exitidx.append(i)
                # capital = stock*close[i]
                
                # brokerage = broker_charge*capital
                # capital -= brokerage
                # total_brokerage+=brokerage
                
                capital += stock*close[entryidx[-1]] - stock*close[i]
                
                portfolio.append(capital)
                # pl = stock * (close[entryidx[-1]] - close[i])
                # gprofitloss.append(pl)
                # nprofitloss.append(pl-brokerage)
                current = 0
                stock=0
                
    
    # if current!=0:
    #     portfolio = portfolio[:entryidx[-1]] + (len(data)-entryidx[-1])*[portfolio[entryidx[-1]-1]]
    #     tradetype.pop()
    #     entryidx.pop()

    df['Signals'] = finalsignals




############################
    return df









data_p = calculate_moving_averages(data_p)
data_p = generate_signals(data_p)


############################
